In [1]:
from torchvision import datasets, models, transforms
import torchvision
import torch
import torch.nn as nn
import copy

In [2]:
# 下面两个框内代码是为了保证.py更新能及时同步到.ipynb中
import sys
import importlib
import dataset_pre
import finetune_model
sys.path.append('./dataset_pre.py')
sys.path.append('./finetune_model.py')

In [3]:
%load_ext autoreload
%autoreload 1
%aimport dataset_pre
%aimport finetune_model
importlib.reload(dataset_pre)
importlib.reload(finetune_model)

<module 'finetune_model' from '/data/lab/AIb_Project/finetune_model.py'>

### Hyperparameters
_____________________

In [4]:
# random seed
SEED = 1

# Dataset class num
NUM_CLASSES_CHEST = 2

# Training
BATCH_SIZE = 32  ##如果是flowers102,batch_size设到64及以上会在训练的第一个epoch爆内存
SAVE_DIR = './log'

### Dataset
_________

In [5]:
data_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#### ChestX

In [6]:
dataset_pre.init_chestX()

In [7]:
chest_train = "./data/chestX/train"
chest_test = "./data/chestX/test"
test_set = torchvision.datasets.ImageFolder(root=chest_test,transform=data_transforms)
test_dataloader = torch.utils.data.DataLoader(dataset=test_set,batch_size=BATCH_SIZE,shuffle=False)

##### Extract data for few-shot

In [8]:
base_folder = "./data/chestX/" 
shot_folder = '5_shot_dataset'
images_per_class = 5

In [9]:
dataset_pre.split_shot_data(chest_train, base_folder+shot_folder, images_per_class)

In [10]:
five_shot_set = torchvision.datasets.ImageFolder(root=base_folder+shot_folder,transform=data_transforms)
five_shot_dataloader = torch.utils.data.DataLoader(dataset=five_shot_set,batch_size=BATCH_SIZE,shuffle=False)

### Model Layer Dimension
___________

In [11]:
# 加载预训练的VGG模型
model = models.vgg16(pretrained=True)

# 模型原最后一层输出类别数量
num_features_out = model.classifier[6].out_features

# 模型原最后一层输入类别数量
num_features_in = model.classifier[6].in_features

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Loss Function
__________________________________

In [12]:
criterion = nn.CrossEntropyLoss()

### Device
___________

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
print(device)

cuda:0


### Finetune
_____________

#### Fixed feature extractor

In [26]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.fixed_feature_extracter(model, num_features_out, 
                                                                     NUM_CLASSES_CHEST, 1e-2)
model = model.to(device)

In [29]:
model_new_linear_layer = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                                    five_shot_set, five_shot_dataloader, test_set, test_dataloader, 10)

Epoch 1/10
----------
Epoch: 1/10 Train Loss: 12.8332
Begin test......
Test Loss: 8.4563,Test Accuracy: 0.3910
Epoch 2/10
----------
Epoch: 2/10 Train Loss: 8.1812
Begin test......
Test Loss: 8.4254,Test Accuracy: 0.6250
Epoch 3/10
----------
Epoch: 3/10 Train Loss: 13.3396
Begin test......
Test Loss: 6.2152,Test Accuracy: 0.6250
Epoch 4/10
----------
Epoch: 4/10 Train Loss: 9.9412
Begin test......
Test Loss: 13.0737,Test Accuracy: 0.3990
Epoch 5/10
----------
Epoch: 5/10 Train Loss: 9.3183
Begin test......
Test Loss: 14.8315,Test Accuracy: 0.4054
Epoch 6/10
----------
Epoch: 6/10 Train Loss: 9.2610
Begin test......
Test Loss: 9.4064,Test Accuracy: 0.4615
Epoch 7/10
----------
Epoch: 7/10 Train Loss: 6.9552
Begin test......
Test Loss: 2.5570,Test Accuracy: 0.6811
Epoch 8/10
----------
Epoch: 8/10 Train Loss: 3.2471
Begin test......
Test Loss: 3.7154,Test Accuracy: 0.7163
Epoch 9/10
----------
Epoch: 9/10 Train Loss: 5.4935
Begin test......
Test Loss: 4.6009,Test Accuracy: 0.7019
Epoch 

#### All fit

1. all fit linear classifier

In [32]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.all_fit("linear", model, num_features_in, NUM_CLASSES_CHEST, 1e-3)
                                                                    
model = model.to(device)
model_before_finetune = copy.deepcopy(model)

In [33]:
model_change_linear = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                                 five_shot_set, five_shot_dataloader, test_set, test_dataloader, 10)

Epoch 1/10
----------
Epoch: 1/10 Train Loss: 0.7465
Begin test......
Test Loss: 0.7254,Test Accuracy: 0.4375
Epoch 2/10
----------
Epoch: 2/10 Train Loss: 0.6242
Begin test......
Test Loss: 0.7524,Test Accuracy: 0.4231
Epoch 3/10
----------
Epoch: 3/10 Train Loss: 0.6057
Begin test......
Test Loss: 0.7719,Test Accuracy: 0.4119
Epoch 4/10
----------
Epoch: 4/10 Train Loss: 0.7005
Begin test......
Test Loss: 0.7854,Test Accuracy: 0.4151
Epoch 5/10
----------
Epoch: 5/10 Train Loss: 0.6870
Begin test......
Test Loss: 0.7797,Test Accuracy: 0.4487
Epoch 6/10
----------
Epoch: 6/10 Train Loss: 0.6759
Begin test......
Test Loss: 0.7573,Test Accuracy: 0.4631
Epoch 7/10
----------
Epoch: 7/10 Train Loss: 0.7769
Begin test......
Test Loss: 0.7174,Test Accuracy: 0.4952
Epoch 8/10
----------
Epoch: 8/10 Train Loss: 0.4527
Begin test......
Test Loss: 0.6736,Test Accuracy: 0.5897
Epoch 9/10
----------
Epoch: 9/10 Train Loss: 0.5088
Begin test......
Test Loss: 0.6507,Test Accuracy: 0.6202
Epoch 10/1

In [34]:
param_changes = finetune_model.compare_params(model_before_finetune, model_change_linear)
# 打印每一层参数的改变率
for name, change_percentage in param_changes.items():
    print(f"Layer: {name}, Change Percentage: {change_percentage:.4f}")
## 这个结论怎么办？为什么会这样？

Layer: features.0.weight, Change Percentage: 0.0012
Layer: features.0.bias, Change Percentage: 0.0007
Layer: features.2.weight, Change Percentage: 0.0013
Layer: features.2.bias, Change Percentage: 0.0009
Layer: features.5.weight, Change Percentage: 0.0008
Layer: features.5.bias, Change Percentage: 0.0006
Layer: features.7.weight, Change Percentage: 0.0009
Layer: features.7.bias, Change Percentage: 0.0004
Layer: features.10.weight, Change Percentage: 0.0009
Layer: features.10.bias, Change Percentage: 0.0003
Layer: features.12.weight, Change Percentage: 0.0009
Layer: features.12.bias, Change Percentage: 0.0002
Layer: features.14.weight, Change Percentage: 0.0009
Layer: features.14.bias, Change Percentage: 0.0002
Layer: features.17.weight, Change Percentage: 0.0010
Layer: features.17.bias, Change Percentage: 0.0001
Layer: features.19.weight, Change Percentage: 0.0011
Layer: features.19.bias, Change Percentage: 0.0001
Layer: features.21.weight, Change Percentage: 0.0011
Layer: features.21.

2. all fit mean centroid classifier

In [14]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.all_fit("mean_centroid", model, num_features_in, 
                                                     NUM_CLASSES_CHEST, 1e-1)
                                                                    
model = model.to(device)

In [15]:
model_change_centroid = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                                 five_shot_set, five_shot_dataloader, test_set, test_dataloader, 25)

Epoch 1/25
----------
Epoch: 1/25 Train Loss: 0.6960
Begin test......
Test Loss: 0.6952,Test Accuracy: 0.4006
Epoch 2/25
----------
Epoch: 2/25 Train Loss: 0.6895
Begin test......
Test Loss: 0.6943,Test Accuracy: 0.4423
Epoch 3/25
----------
Epoch: 3/25 Train Loss: 0.6925
Begin test......
Test Loss: 0.6938,Test Accuracy: 0.4647
Epoch 4/25
----------
Epoch: 4/25 Train Loss: 0.6866
Begin test......
Test Loss: 0.6919,Test Accuracy: 0.5561
Epoch 5/25
----------
Epoch: 5/25 Train Loss: 0.6927
Begin test......
Test Loss: 0.6934,Test Accuracy: 0.4503
Epoch 6/25
----------
Epoch: 6/25 Train Loss: 0.6943
Begin test......
Test Loss: 0.6936,Test Accuracy: 0.4455
Epoch 7/25
----------
Epoch: 7/25 Train Loss: 0.6870
Begin test......
Test Loss: 0.6949,Test Accuracy: 0.3830
Epoch 8/25
----------
Epoch: 8/25 Train Loss: 0.6833
Begin test......
Test Loss: 0.6940,Test Accuracy: 0.4006
Epoch 9/25
----------
Epoch: 9/25 Train Loss: 0.6909
Begin test......
Test Loss: 0.6945,Test Accuracy: 0.3926
Epoch 10/2

3. all fit cosine classifier

In [16]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.all_fit("cosine", model, num_features_in, 
                                                     NUM_CLASSES_CHEST, 1e-1)
                                                                    
model = model.to(device)

In [17]:
model_change_cosine = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                                 five_shot_set, five_shot_dataloader, test_set, test_dataloader, 25)

Epoch 1/25
----------
Epoch: 1/25 Train Loss: 0.6935
Begin test......
Test Loss: 0.6928,Test Accuracy: 0.5096
Epoch 2/25
----------
Epoch: 2/25 Train Loss: 0.6945
Begin test......
Test Loss: 0.6921,Test Accuracy: 0.5657
Epoch 3/25
----------
Epoch: 3/25 Train Loss: 0.6920
Begin test......
Test Loss: 0.6910,Test Accuracy: 0.6138
Epoch 4/25
----------
Epoch: 4/25 Train Loss: 0.6984
Begin test......
Test Loss: 0.6894,Test Accuracy: 0.6571
Epoch 5/25
----------
Epoch: 5/25 Train Loss: 0.6865
Begin test......
Test Loss: 0.6874,Test Accuracy: 0.7051
Epoch 6/25
----------
Epoch: 6/25 Train Loss: 0.6878
Begin test......
Test Loss: 0.6873,Test Accuracy: 0.6795
Epoch 7/25
----------
Epoch: 7/25 Train Loss: 0.6828
Begin test......
Test Loss: 0.6869,Test Accuracy: 0.7115
Epoch 8/25
----------
Epoch: 8/25 Train Loss: 0.6847
Begin test......
Test Loss: 0.6871,Test Accuracy: 0.6843
Epoch 9/25
----------
Epoch: 9/25 Train Loss: 0.6925
Begin test......
Test Loss: 0.6862,Test Accuracy: 0.7244
Epoch 10/2

#### Fit last k(1,2,3)

1. fit last1

In [14]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.fit_last(model, num_features_in, 
                                                     NUM_CLASSES_CHEST, 1e-2, 1)

model = model.to(device)

In [16]:
model_fit_l1 = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                          five_shot_set, five_shot_dataloader, test_set, test_dataloader, 10)

Epoch 1/10
----------
Epoch: 1/10 Train Loss: 0.9316
Begin test......
Test Loss: 0.7089,Test Accuracy: 0.5288
Epoch 2/10
----------
Epoch: 2/10 Train Loss: 0.7640
Begin test......
Test Loss: 0.8438,Test Accuracy: 0.6250
Epoch 3/10
----------
Epoch: 3/10 Train Loss: 1.0760
Begin test......
Test Loss: 0.7606,Test Accuracy: 0.6122
Epoch 4/10
----------
Epoch: 4/10 Train Loss: 0.9286
Begin test......
Test Loss: 1.0769,Test Accuracy: 0.5128
Epoch 5/10
----------
Epoch: 5/10 Train Loss: 0.8411
Begin test......
Test Loss: 0.8211,Test Accuracy: 0.6234
Epoch 6/10
----------
Epoch: 6/10 Train Loss: 0.2919
Begin test......
Test Loss: 0.7006,Test Accuracy: 0.6811
Epoch 7/10
----------
Epoch: 7/10 Train Loss: 0.5583
Begin test......
Test Loss: 0.7335,Test Accuracy: 0.7003
Epoch 8/10
----------
Epoch: 8/10 Train Loss: 0.9337
Begin test......
Test Loss: 0.6418,Test Accuracy: 0.7179
Epoch 9/10
----------
Epoch: 9/10 Train Loss: 0.1171
Begin test......
Test Loss: 0.8696,Test Accuracy: 0.6346
Epoch 10/1

2. fit last2

In [19]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.fit_last(model, num_features_in, 
                                                     NUM_CLASSES_CHEST, 1e-2, 2)

model = model.to(device)

In [20]:
model_fit_l2 = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                          five_shot_set, five_shot_dataloader, test_set, test_dataloader, 15)

Epoch 1/15
----------
Epoch: 1/15 Train Loss: 0.6473
Begin test......
Test Loss: 0.7664,Test Accuracy: 0.3766
Epoch 2/15
----------
Epoch: 2/15 Train Loss: 0.6774
Begin test......
Test Loss: 0.7225,Test Accuracy: 0.4215
Epoch 3/15
----------
Epoch: 3/15 Train Loss: 0.6726
Begin test......
Test Loss: 0.6834,Test Accuracy: 0.5721
Epoch 4/15
----------
Epoch: 4/15 Train Loss: 0.6684
Begin test......
Test Loss: 0.6631,Test Accuracy: 0.6090
Epoch 5/15
----------
Epoch: 5/15 Train Loss: 0.6416
Begin test......
Test Loss: 0.6525,Test Accuracy: 0.6298
Epoch 6/15
----------
Epoch: 6/15 Train Loss: 0.5414
Begin test......
Test Loss: 0.6437,Test Accuracy: 0.6250
Epoch 7/15
----------
Epoch: 7/15 Train Loss: 0.6116
Begin test......
Test Loss: 0.6428,Test Accuracy: 0.6298
Epoch 8/15
----------
Epoch: 8/15 Train Loss: 0.5952
Begin test......
Test Loss: 0.6680,Test Accuracy: 0.5849
Epoch 9/15
----------
Epoch: 9/15 Train Loss: 0.4385
Begin test......
Test Loss: 0.6657,Test Accuracy: 0.5881
Epoch 10/1

3. fit last3

In [23]:
model = models.vgg16(pretrained=True)
model, optimizer, scheduler = finetune_model.fit_last(model, num_features_in, 
                                                     NUM_CLASSES_CHEST, 1e-1, 3)

model = model.to(device)

In [24]:
model_fit_l3 = finetune_model.train_model(model, device, criterion, optimizer, scheduler, 
                                          five_shot_set, five_shot_dataloader, test_set, test_dataloader, 15)

Epoch 1/15
----------
Epoch: 1/15 Train Loss: 0.6723
Begin test......
Test Loss: 0.7077,Test Accuracy: 0.4792
Epoch 2/15
----------
Epoch: 2/15 Train Loss: 0.4992
Begin test......
Test Loss: 0.6944,Test Accuracy: 0.5304
Epoch 3/15
----------
Epoch: 3/15 Train Loss: 0.4355
Begin test......
Test Loss: 0.6969,Test Accuracy: 0.5449
Epoch 4/15
----------
Epoch: 4/15 Train Loss: 0.5596
Begin test......
Test Loss: 0.7227,Test Accuracy: 0.5337
Epoch 5/15
----------
Epoch: 5/15 Train Loss: 0.6612
Begin test......
Test Loss: 0.6501,Test Accuracy: 0.5946
Epoch 6/15
----------
Epoch: 6/15 Train Loss: 0.7079
Begin test......
Test Loss: 0.6533,Test Accuracy: 0.6122
Epoch 7/15
----------
Epoch: 7/15 Train Loss: 0.7235
Begin test......
Test Loss: 0.6427,Test Accuracy: 0.6571
Epoch 8/15
----------
Epoch: 8/15 Train Loss: 0.6620
Begin test......
Test Loss: 0.6582,Test Accuracy: 0.6074
Epoch 9/15
----------
Epoch: 9/15 Train Loss: 0.6426
Begin test......
Test Loss: 0.6558,Test Accuracy: 0.6250
Epoch 10/1